In [2]:
# check if pyspark works
import pyspark
print(pyspark.__version__)

3.5.1


In [3]:
from pyspark.sql import SparkSession

# Start spark session
spark = SparkSession.builder \
    .appName("Multiclass classification IoT") \
    .getOrCreate()

# Load datasets
def load_test_data(debug=False):
    if debug:
        test_data = spark.read.csv(r"./dataset/NF-ToN-IoT-v2-test.csv", header=True, inferSchema=True).limit(200)
        train_data = spark.read.csv(r"./dataset/NF-ToN-IoT-v2-train.csv", header=True, inferSchema=True).limit(300000)
    else:
        test_data = spark.read.csv(r"./dataset/NF-ToN-IoT-v2-test.csv", header=True, inferSchema=True)
        train_data = spark.read.csv(r"./dataset/NF-ToN-IoT-v2-train.csv", header=True, inferSchema=True)
        
    return test_data, train_data

test_data, train_data = load_test_data()


In [ ]:
# Display the first few rows of the training dataset to verify it's loaded correctly
train_data.show(5)
print(train_data.count())

# Display the first few rows of the testing dataset to verify it's loaded correctly
test_data.show(5)
print(test_data.count())

In [ ]:
# Print the schema of the loaded data to confirm the data types of each column
train_data.printSchema()
test_data.printSchema()

### Data Preprocessing and Exploratory Data Analysis (EDA)

In [ ]:
# Identifying missing values
from pyspark.sql.functions import col, count, when, isnan

# Count nulls and NaNs in each column
def count_missing(data):
    #data.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in data.columns]).show()
    data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()

count_missing(train_data)

In [ ]:
# Identifying missing values
#count_missing(test_data)

The dataset is clean and doesn't have any missing values, so there's no need for further cleaning.

#### Encoding
On the dataset, which consists of network traffic data, several categorical variables could potentially benefit from encoding. The decision to apply encoding techniques depends on whether the variables are nominal (without an inherent order) or ordinal (with a specific order) and whether they are used as features in the model. 

After an understanding of data and the variables meaning, we considered some variables for encoding:
- **IPV4_SRC_ADDR and IPV4_DST_ADDR** (Categorical nominal): These are IP addresses and typically should be treated as categorical.
- **Attack** (categorical Nominal): In machine learning projects involving classification tasks, the target variable (also known as the label or response variable) is crucial as it's the outcome the model is trying to predict. On the project, the target variable consists of categorical data (e.g., text labels representing different classes), it needs to be converted into a numeric format. This conversion is essential because most machine learning algorithms require numeric input to perform calculations during model training.


In [ ]:
from pyspark.sql.functions import countDistinct

# Count distinct values in IPV4_SRC_ADDR
distinct_src_subnet_count = train_data.select(countDistinct("IPV4_SRC_ADDR").alias("Distinct_SRC_Count"))

# Count distinct values in IPV4_DST_ADDR
distinct_dst_subnet_count = train_data.select(countDistinct("IPV4_DST_ADDR").alias("Distinct_DST_Subnet"))

# Show the results
distinct_src_subnet_count.show(truncate=False)
distinct_dst_subnet_count.show(truncate=False)

Given the high number of distinct values in the IPV4_ADDR column, subnet segmentation was performed to facilitate more effective encoding. This approach reduces the granularity of the data, thereby simplifying the feature space without significantly compromising the informational value of the IP addresses. Segmentation enables us to manage the high cardinality of the IP addresses, which is critical for applying machine learning techniques efficiently and effectively. By categorizing the IP addresses into their respective subnets, we can capture essential network-level behaviors while avoiding the computational complexity associated with the vast number of unique full IP addresses.

In [5]:
from pyspark.sql.functions import concat_ws, split, col

def add_subnet_columns(data):
    # Add a new column for the subnet (first two octets) for source and destination IP addresses
    data = data.withColumn(
        "IPV4_SRC_ADDR_Subnet",
        concat_ws(".", split(col("IPV4_SRC_ADDR"), "\\.")[0], split(col("IPV4_SRC_ADDR"), "\\.")[1])
    )

    data = data.withColumn(
        "IPV4_DST_ADDR_Subnet",
        concat_ws(".", split(col("IPV4_DST_ADDR"), "\\.")[0], split(col("IPV4_DST_ADDR"), "\\.")[1])
    )

    return data

train_data = add_subnet_columns(train_data)
test_data = add_subnet_columns(test_data)

In [ ]:
# Show the new columns along with the original IP addresses
train_data.select("IPV4_SRC_ADDR", "IPV4_SRC_ADDR_Subnet", "IPV4_DST_ADDR", "IPV4_DST_ADDR_Subnet").show(truncate=False)

In [ ]:
from pyspark.sql.functions import countDistinct

# Count distinct values in IPV4_SRC_ADDR_Subnet
distinct_src_subnet_count = train_data.select(countDistinct("IPV4_SRC_ADDR_Subnet").alias("Distinct_SRC_Subnet_Count"))

# Count distinct values in IPV4_DST_ADDR_Subnet
distinct_dst_subnet_count = train_data.select(countDistinct("IPV4_DST_ADDR_Subnet").alias("Distinct_DST_Subnet_Count"))

# Show the results
distinct_src_subnet_count.show(truncate=False)
distinct_dst_subnet_count.show(truncate=False)

- **Source IP Dimensionality:** The modest reduction in unique source subnets compared to the full source IP count suggests that the network traffic originates from a diverse set of locations or devices, with slightly clustered but still quite spread out origins. This might imply that any predictive modeling using source subnets as a feature would still need to handle a relatively high number of categories, potentially requiring further techniques to manage dimensionality or enhance interpretability.

- **Destination IP Dimensionality:** The more significant reduction in unique destination subnets points to a higher level of concentration of network traffic towards certain destination networks or servers. For modeling purposes, this could mean that destination subnet could be a more impactful feature, providing stronger predictive signals with fewer categories, thus improving model performance and simplicity.

Given the results, for destination subnets, straightforward categorical encoding methods like one-hot encoding might be feasible given the reduced number of unique values. For source subnets, considering the still high number of unique categories, methods like feature hashing or embedding might be more appropriate to prevent models from becoming too complex.

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, FeatureHasher
from pyspark.ml import Pipeline

# Define Indexers and Encoders for categorical variables
attack_indexer = StringIndexer(inputCol="Attack", outputCol="attack_index")

# Destination address
dst_subnet_indexer = StringIndexer(inputCol="IPV4_DST_ADDR_Subnet", outputCol="dst_subnet_index")

# Source address
# Feature hashing for source subnet since lower cardinality
# Just for graphing
src_subnet_indexer = StringIndexer(inputCol="IPV4_SRC_ADDR_Subnet", outputCol="src_subnet_index")


# Create a pipeline for encoding
encoding_pipeline = Pipeline(stages=[
    attack_indexer, 
    dst_subnet_indexer, 
    src_subnet_indexer,
])

# Fit the encoding pipeline to the training data
encoding_model = encoding_pipeline.fit(train_data)

# Transform both training and test data
encoded_train_data = encoding_model.transform(train_data)
encoded_test_data = encoding_model.transform(test_data)

# Show encoded features to verify
encoded_train_data.select("Attack", "attack_index", "IPV4_SRC_ADDR_Subnet", "IPV4_DST_ADDR_Subnet", "dst_subnet_index").show(truncate=False)

+---------+------------+--------------------+--------------------+----------------+
|Attack   |attack_index|IPV4_SRC_ADDR_Subnet|IPV4_DST_ADDR_Subnet|dst_subnet_index|
+---------+------------+--------------------+--------------------+----------------+
|scanning |1.0         |192.168             |192.168             |0.0             |
|scanning |1.0         |192.168             |192.168             |0.0             |
|Benign   |0.0         |192.168             |192.168             |0.0             |
|scanning |1.0         |192.168             |192.168             |0.0             |
|ddos     |3.0         |192.168             |192.168             |0.0             |
|Benign   |0.0         |192.168             |192.168             |0.0             |
|Benign   |0.0         |192.168             |192.168             |0.0             |
|password |4.0         |192.168             |192.168             |0.0             |
|xss      |2.0         |192.168             |192.168             |0.0       

In [7]:
# Drop the original columns that have been encoded and hashed
encoded_train_data = encoded_train_data.drop("Attack", "Label", "IPV4_SRC_ADDR_Subnet", "IPV4_DST_ADDR_Subnet", "IPV4_SRC_ADDR", "IPV4_DST_ADDR")

# Show the final columns in the training data
encoded_train_data.columns

['L4_SRC_PORT',
 'L4_DST_PORT',
 'PROTOCOL',
 'L7_PROTO',
 'IN_BYTES',
 'IN_PKTS',
 'OUT_BYTES',
 'OUT_PKTS',
 'TCP_FLAGS',
 'CLIENT_TCP_FLAGS',
 'SERVER_TCP_FLAGS',
 'FLOW_DURATION_MILLISECONDS',
 'DURATION_IN',
 'DURATION_OUT',
 'MIN_TTL',
 'MAX_TTL',
 'LONGEST_FLOW_PKT',
 'SHORTEST_FLOW_PKT',
 'MIN_IP_PKT_LEN',
 'MAX_IP_PKT_LEN',
 'SRC_TO_DST_SECOND_BYTES',
 'DST_TO_SRC_SECOND_BYTES',
 'RETRANSMITTED_IN_BYTES',
 'RETRANSMITTED_IN_PKTS',
 'RETRANSMITTED_OUT_BYTES',
 'RETRANSMITTED_OUT_PKTS',
 'SRC_TO_DST_AVG_THROUGHPUT',
 'DST_TO_SRC_AVG_THROUGHPUT',
 'NUM_PKTS_UP_TO_128_BYTES',
 'NUM_PKTS_128_TO_256_BYTES',
 'NUM_PKTS_256_TO_512_BYTES',
 'NUM_PKTS_512_TO_1024_BYTES',
 'NUM_PKTS_1024_TO_1514_BYTES',
 'TCP_WIN_MAX_IN',
 'TCP_WIN_MAX_OUT',
 'ICMP_TYPE',
 'ICMP_IPV4_TYPE',
 'DNS_QUERY_ID',
 'DNS_QUERY_TYPE',
 'DNS_TTL_ANSWER',
 'FTP_COMMAND_RET_CODE',
 'attack_index',
 'dst_subnet_index',
 'src_subnet_index']

# EDA part
Converting dataset to pandas for graphics

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Row limit for converting to Pandas DataFrame for plotting (to avoid memory issues)
ROW_LIMIT = 200000

# Converte il DataFrame Spark in un DataFrame Pandas per l'analisi esplorativa
df_pd = encoded_train_data.limit(ROW_LIMIT).toPandas()


In [ ]:
# Analisi esplorativa dei dati utilizzando Pandas e Seaborn
def features_hisogram(dataframe):
    # Istogrammi delle caratteristiche
    dataframe.hist(bins=20, figsize=(96, 48))
    plt.suptitle("Histograms of Features", fontsize=16)
    plt.tight_layout()
    plt.show()
    
features_hisogram(df_pd)

In [ ]:
def heatmap(dataframe):
    # Matrice di correlazione
    correlation_matrix = dataframe.corr()
    plt.figure(figsize=(48, 48))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix', fontsize=20)
    plt.show()
    
heatmap(df_pd)

In [ ]:
# Frequencias de categoria
# This function calculates the frequency of each category in a given column
# It use the full dataset to calculate the frequencies

from pyspark.sql.functions import count

def category_frequencies(data, column):
    # Calcular a frequência das categorias na variável 'Attack'
    frequency = data.groupBy(column).agg(count("*").alias("frequency")).orderBy("frequency", ascending=False)
    #attack_freq.show(truncate=False)
    return frequency

category_frequencies(encoded_train_data, "attack_index").show(truncate=False)



In [ ]:
# Frequencias de categoria (plot)
import matplotlib.pyplot as plt
import pandas as pd

# This function plots the top categories in a given column
# It reduces the dataset to the top ROW_LIMIT categories before plotting

def plot_category_frequencies(data, column):
    data = category_frequencies(data, column).limit(ROW_LIMIT).toPandas()
    plt.figure(figsize=(10, 6))
    plt.bar(data[column], data['frequency'], color='blue')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.title(f'Top {ROW_LIMIT} Frequency of {column} Categories')
    plt.xticks(rotation=90)
    plt.show()

plot_category_frequencies(encoded_train_data, "attack_index")



# PCA analysis

In [8]:
COMPONENTS = 3

TARGET = "attack_index"

FEATURES = [
    "L4_SRC_PORT",
    "L4_DST_PORT",
    "ICMP_TYPE",
    "ICMP_IPV4_TYPE",
    "DNS_QUERY_ID",
    "DNS_QUERY_TYPE",
    "DNS_TTL_ANSWER",
    "FTP_COMMAND_RET_CODE",
    "IN_BYTES",
    "OUT_BYTES",
    "IN_PKTS",
    "OUT_PKTS",
    "FLOW_DURATION_MILLISECONDS",
    "PROTOCOL",
    "SRC_TO_DST_SECOND_BYTES",
    "DST_TO_SRC_SECOND_BYTES",
    "LONGEST_FLOW_PKT",
    "SHORTEST_FLOW_PKT",
    "MIN_IP_PKT_LEN",
    "MAX_IP_PKT_LEN",
    "TCP_WIN_MAX_IN",
    "TCP_WIN_MAX_OUT",
    "dst_subnet_index",
    "src_subnet_index",
    "CLIENT_TCP_FLAGS",
    "SERVER_TCP_FLAGS",
    "DURATION_OUT",
    "DURATION_IN",
    "MIN_TTL",
    "NUM_PKTS_UP_TO_128_BYTES",
    "RETRANSMITTED_IN_BYTES",
    "MAX_TTL",
    "NUM_PKTS_128_TO_256_BYTES",
    "DST_TO_SRC_AVG_THROUGHPUT",
    "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES",
    "RETRANSMITTED_OUT_BYTES",
    "RETRANSMITTED_OUT_PKTS",
    "RETRANSMITTED_IN_PKTS",
    "SRC_TO_DST_AVG_THROUGHPUT",
    "L7_PROTO",
    "NUM_PKTS_1024_TO_1514_BYTES",
    "TCP_FLAGS",
]

# Define the features
FEATURES = [
    "L4_SRC_PORT", "L4_DST_PORT", "ICMP_TYPE", "ICMP_IPV4_TYPE", "DNS_QUERY_ID", "DNS_QUERY_TYPE",
    "DNS_TTL_ANSWER", "FTP_COMMAND_RET_CODE", "IN_BYTES", "OUT_BYTES", "IN_PKTS", "OUT_PKTS",
    "FLOW_DURATION_MILLISECONDS", "PROTOCOL", "SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES",
    "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "TCP_WIN_MAX_IN",
    "TCP_WIN_MAX_OUT", "dst_subnet_index", "src_subnet_index", "CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS",
    "DURATION_OUT", "DURATION_IN", "MIN_TTL", "NUM_PKTS_UP_TO_128_BYTES", "RETRANSMITTED_IN_BYTES",
    "MAX_TTL", "NUM_PKTS_128_TO_256_BYTES", "DST_TO_SRC_AVG_THROUGHPUT", "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES", "RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS", "RETRANSMITTED_IN_PKTS",
    "SRC_TO_DST_AVG_THROUGHPUT", "L7_PROTO", "NUM_PKTS_1024_TO_1514_BYTES", "TCP_FLAGS"
]


feature_groups = {
    "trasimitted_in": ["IN_BYTES", "IN_PKTS"],
    "trasmitted_out": ["OUT_BYTES", "OUT_PKTS"],
    "retransimtted_in": ["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"],
    "retransimtted_out": ["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"],
    "throughput_in": ["SRC_TO_DST_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT"],
    "throughput_out": ["DST_TO_SRC_AVG_THROUGHPUT", "DST_TO_SRC_SECOND_BYTES"],
    "ttl": ["MIN_TTL", "MAX_TTL"],
    "tcp_flags": ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"],
    "icmp": ["ICMP_TYPE", "ICMP_IPV4_TYPE"],
    "flow_max": ["LONGEST_FLOW_PKT", "MAX_IP_PKT_LEN"],
    "flow_min": ["SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN"],
    "flow_duration": ["FLOW_DURATION_MILLISECONDS"],
    "tcp_in": ["TCP_WIN_MAX_IN"],
    "tcp_out": ["TCP_WIN_MAX_OUT"],
    "duration_in": ["DURATION_IN"],
    "duration_out": ["DURATION_OUT"],
    "protocol": ["PROTOCOL"],
    "l4_src_port": ["L4_SRC_PORT"],
    "l4_dst_port": ["L4_DST_PORT"],
    "dns": ["DNS_QUERY_ID", "DNS_QUERY_TYPE", "DNS_TTL_ANSWER"],
    "ftp": ["FTP_COMMAND_RET_CODE"],
    "l7_proto": ["L7_PROTO"],
    "num_packets_by_size": ["NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES"]
}

feature_groups = {
    "ttl": ["MIN_TTL", "MAX_TTL"],
    "tcp_flags": ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"],
    "flow": ["LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT"],
    "duration" : ["FLOW_DURATION_MILLISECONDS", "DURATION_OUT", "DURATION_IN"],
    "pkt_size": ["MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN"],
    "tcp_win": ["TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT"],
    "throughput": ["SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT", "DST_TO_SRC_AVG_THROUGHPUT"],
    "protocol": ["PROTOCOL"],
    "l4_dst_port": ["L4_DST_PORT"],
    "dns": ["DNS_QUERY_ID", "DNS_QUERY_TYPE"]
}


In [9]:
# Define feature groups
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# OneHot encoding for destination subnet since higher cardinality
dst_subnet_encoder = OneHotEncoder(inputCol="dst_subnet_index", outputCol="dst_subnet_vec")

# Feature hashing for source subnet since lower cardinality
src_subnet_hasher = FeatureHasher(inputCols=["src_subnet_index"], outputCol="src_subnet_hashed", numFeatures=1024)

# Create assemblers and scalers
assemblers_and_scalers = []

def create_assembler_and_scaler(inputCols, outputCol):
    assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)
    scaler = StandardScaler(inputCol=outputCol, outputCol=f"scaled_{outputCol}", withStd=True, withMean=True)
    return assembler, scaler

for group_name, features in feature_groups.items():
    assembler, scaler = create_assembler_and_scaler(features, f"{group_name}_features")
    assemblers_and_scalers.append(assembler)
    assemblers_and_scalers.append(scaler)

# Create the complete pipeline
pipeline = Pipeline(stages=[dst_subnet_encoder, src_subnet_hasher] + assemblers_and_scalers)

# Fit the pipeline and transform the data
pipeline_model = pipeline.fit(encoded_train_data)
processed_train_data = pipeline_model.transform(encoded_train_data)

In [10]:
# Assemble all scaled features into a single feature vector
all_scaled_features = [f"scaled_{group_name}_features" for group_name in feature_groups.keys()]
#all_assebled_features = [f"{group_name}_features" for group_name in feature_groups.keys()]
all_scaled_features = all_scaled_features + ["src_subnet_hashed", "dst_subnet_vec"]

dataset_features_to_keep = all_scaled_features + ["attack_index"]
to_remove = [c for c in processed_train_data.columns if c not in dataset_features_to_keep]
# Save the processed training data removing the intermediate columns
for column in to_remove:
    processed_train_data = processed_train_data.drop(column)
# Show the final columns in the training data
print(processed_train_data.columns)


['attack_index', 'dst_subnet_vec', 'src_subnet_hashed', 'scaled_ttl_features', 'scaled_tcp_flags_features', 'scaled_flow_features', 'scaled_duration_features', 'scaled_pkt_size_features', 'scaled_tcp_win_features', 'scaled_throughput_features', 'scaled_protocol_features', 'scaled_l4_dst_port_features', 'scaled_dns_features']


In [11]:
# Save the dataset into parquet format
processed_train_data.write.mode("overwrite").parquet("scaled_simplified_train_data.parquet")

In [1]:
from pyspark.sql import SparkSession

# Start spark session
spark = SparkSession.builder \
    .appName("Multiclass classification IoT") \
    .getOrCreate()

# Load datasets
def load_test_data(debug=False):
    if debug:
        train_data = spark.read.parquet("scaled_simplified_train_data.parquet").limit(300000)
        
    else:
        train_data = spark.read.parquet("scaled_simplified_train_data.parquet")
        
    return train_data

processed_train_data = load_test_data()

In [2]:
processed_train_data.printSchema()
# List the columns in the processed training data
processed_train_data.columns

root
 |-- attack_index: double (nullable = true)
 |-- dst_subnet_vec: vector (nullable = true)
 |-- src_subnet_hashed: vector (nullable = true)
 |-- scaled_ttl_features: vector (nullable = true)
 |-- scaled_tcp_flags_features: vector (nullable = true)
 |-- scaled_flow_features: vector (nullable = true)
 |-- scaled_duration_features: vector (nullable = true)
 |-- scaled_pkt_size_features: vector (nullable = true)
 |-- scaled_tcp_win_features: vector (nullable = true)
 |-- scaled_throughput_features: vector (nullable = true)
 |-- scaled_protocol_features: vector (nullable = true)
 |-- scaled_l4_dst_port_features: vector (nullable = true)
 |-- scaled_dns_features: vector (nullable = true)



['attack_index',
 'dst_subnet_vec',
 'src_subnet_hashed',
 'scaled_ttl_features',
 'scaled_tcp_flags_features',
 'scaled_flow_features',
 'scaled_duration_features',
 'scaled_pkt_size_features',
 'scaled_tcp_win_features',
 'scaled_throughput_features',
 'scaled_protocol_features',
 'scaled_l4_dst_port_features',
 'scaled_dns_features']

In [3]:
COMPONENTS = 3

TARGET = "attack_index"

FEATURES = [
    "L4_SRC_PORT",
    "L4_DST_PORT",
    "ICMP_TYPE",
    "ICMP_IPV4_TYPE",
    "DNS_QUERY_ID",
    "DNS_QUERY_TYPE",
    "DNS_TTL_ANSWER",
    "FTP_COMMAND_RET_CODE",
    "IN_BYTES",
    "OUT_BYTES",
    "IN_PKTS",
    "OUT_PKTS",
    "FLOW_DURATION_MILLISECONDS",
    "PROTOCOL",
    "SRC_TO_DST_SECOND_BYTES",
    "DST_TO_SRC_SECOND_BYTES",
    "LONGEST_FLOW_PKT",
    "SHORTEST_FLOW_PKT",
    "MIN_IP_PKT_LEN",
    "MAX_IP_PKT_LEN",
    "TCP_WIN_MAX_IN",
    "TCP_WIN_MAX_OUT",
    "dst_subnet_index",
    "src_subnet_index",
    "CLIENT_TCP_FLAGS",
    "SERVER_TCP_FLAGS",
    "DURATION_OUT",
    "DURATION_IN",
    "MIN_TTL",
    "NUM_PKTS_UP_TO_128_BYTES",
    "RETRANSMITTED_IN_BYTES",
    "MAX_TTL",
    "NUM_PKTS_128_TO_256_BYTES",
    "DST_TO_SRC_AVG_THROUGHPUT",
    "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES",
    "RETRANSMITTED_OUT_BYTES",
    "RETRANSMITTED_OUT_PKTS",
    "RETRANSMITTED_IN_PKTS",
    "SRC_TO_DST_AVG_THROUGHPUT",
    "L7_PROTO",
    "NUM_PKTS_1024_TO_1514_BYTES",
    "TCP_FLAGS",
]

# Define the features
FEATURES = [
    "L4_SRC_PORT", "L4_DST_PORT", "ICMP_TYPE", "ICMP_IPV4_TYPE", "DNS_QUERY_ID", "DNS_QUERY_TYPE",
    "DNS_TTL_ANSWER", "FTP_COMMAND_RET_CODE", "IN_BYTES", "OUT_BYTES", "IN_PKTS", "OUT_PKTS",
    "FLOW_DURATION_MILLISECONDS", "PROTOCOL", "SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES",
    "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "TCP_WIN_MAX_IN",
    "TCP_WIN_MAX_OUT", "dst_subnet_index", "src_subnet_index", "CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS",
    "DURATION_OUT", "DURATION_IN", "MIN_TTL", "NUM_PKTS_UP_TO_128_BYTES", "RETRANSMITTED_IN_BYTES",
    "MAX_TTL", "NUM_PKTS_128_TO_256_BYTES", "DST_TO_SRC_AVG_THROUGHPUT", "NUM_PKTS_512_TO_1024_BYTES",
    "NUM_PKTS_256_TO_512_BYTES", "RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS", "RETRANSMITTED_IN_PKTS",
    "SRC_TO_DST_AVG_THROUGHPUT", "L7_PROTO", "NUM_PKTS_1024_TO_1514_BYTES", "TCP_FLAGS"
]


feature_groups = {
    "trasimitted_in": ["IN_BYTES", "IN_PKTS"],
    "trasmitted_out": ["OUT_BYTES", "OUT_PKTS"],
    "retransimtted_in": ["RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS"],
    "retransimtted_out": ["RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS"],
    "throughput_in": ["SRC_TO_DST_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT"],
    "throughput_out": ["DST_TO_SRC_AVG_THROUGHPUT", "DST_TO_SRC_SECOND_BYTES"],
    "ttl": ["MIN_TTL", "MAX_TTL"],
    "tcp_flags": ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"],
    "icmp": ["ICMP_TYPE", "ICMP_IPV4_TYPE"],
    "flow" : ["LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "FLOW_DURATION_MILLISECONDS"],
    "tcp_win": ["TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT"],
    "duration": ["DURATION_OUT", "DURATION_IN"],
    "protocol": ["PROTOCOL"],
    "l4_src_port": ["L4_SRC_PORT"],
    "l4_dst_port": ["L4_DST_PORT"],
    "dns": ["DNS_QUERY_ID", "DNS_QUERY_TYPE", "DNS_TTL_ANSWER"],
    "ftp": ["FTP_COMMAND_RET_CODE"],
    "num_packets_by_size": ["NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES"]
}

feature_groups = {
    "ttl": ["MIN_TTL", "MAX_TTL"],
    "tcp_flags": ["CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", "TCP_FLAGS"],
    "flow": ["LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT"],
    "duration" : ["FLOW_DURATION_MILLISECONDS", "DURATION_OUT", "DURATION_IN"],
    "pkt_size": ["MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN"],
    "tcp_win": ["TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT"],
    "throughput": ["SRC_TO_DST_SECOND_BYTES", "DST_TO_SRC_SECOND_BYTES", "SRC_TO_DST_AVG_THROUGHPUT", "DST_TO_SRC_AVG_THROUGHPUT"],
    "protocol": ["PROTOCOL"],
    "l4_dst_port": ["L4_DST_PORT"],
    "dns": ["DNS_QUERY_ID", "DNS_QUERY_TYPE"]
}


all_scaled_features = [f"scaled_{group_name}_features" for group_name in feature_groups.keys()]
all_assebled_features = [f"{group_name}_features" for group_name in feature_groups.keys()]

In [4]:
# Perform PCA
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

components = 2
final_assembler = VectorAssembler(inputCols=all_scaled_features + ["dst_subnet_vec", "src_subnet_hashed"], outputCol="final_features")
pca = PCA(k=components, inputCol="final_features", outputCol="pca_features")
pipeline = Pipeline(stages=[final_assembler, pca])


# Assemble all scaled features into a single feature vector
#pca_model, pca_result = perform_pca(processed_train_data, components=COMPONENTS)


In [5]:
pca_model = pipeline.fit(processed_train_data)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/pyspark/errors/exceptions/captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <exception str() failed>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/vscode_pyspark/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, 

Py4JError: py4j.reflection.TypeUtil.isInstanceOf does not exist in the JVM

In [ ]:
pca_result = pca_model.transform(processed_train_data)

In [ ]:
# Extract principal components from vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType



# Show the principal components
def extract_component(vector, index):
    return float(vector[index])

for i in range(COMPONENTS):
    processed_train_data = pca_model.withColumn(f'PC{i+1}', udf(lambda vector: extract_component(vector, i), FloatType())(col('pca_features')))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_pca_scatter(
    pca_result, target: str = TARGET, num_components: int = COMPONENTS
):

    # Convert PCA results to Pandas DataFrame for visualization
    pca_pd = pca_result.limit(ROW_LIMIT).toPandas()

    # Extract principal components into separate columns
    pca_columns = [f"PC{i+1}" for i in range(num_components)]
    pca_components = pd.DataFrame(
        pca_pd["pca_features"].apply(lambda x: x[:num_components]).tolist(),
        columns=pca_columns,
        index=pca_pd.index,
    )

    # Combine PCA components with the target variable
    pca_pd = pd.concat([pca_pd, pca_components], axis=1)

    def scatter_plot(pca_pd, x, y):
        plt.figure(figsize=(6, 6))
        sns.scatterplot(x=x, y=y, hue=target, data=pca_pd, palette="viridis", alpha=0.7)
        plt.xlabel(x)
        plt.ylabel(y)
        plt.title(f"PCA - {x} vs {y}")
        plt.legend(title=target)
        plt.show()

    # Plot foreach pair of principal components
    for i in range(num_components):
        for j in range(i + 1, num_components):
            scatter_plot(pca_pd, pca_columns[i], pca_columns[j])


# Plot PCA results
plot_pca_scatter(pca_result)

In [ ]:
def significant_features_by_pca(pca_model, features = all_scaled_features):    
    # Estrai il modello PCA
    pca_stage = pca_model.stages[-1]
    
    # Ottieni le componenti principali
    pca_components = pca_stage.pc.toArray()
    
    # Ottieni la varianza spiegata
    explained_variance = pca_stage.explainedVariance.toArray()
    
    # Determina le features più significative
    significant_features = {}
    for i, ev in enumerate(explained_variance):
        component = pca_components[:, i]
        # Le features significative per questa componente
        feature_contributions = {features[j]: abs(component[j]) for j in range(len(features))}
        # Ordina per contributo in ordine decrescente
        sorted_features = sorted(feature_contributions.items(), key=lambda x: x[1], reverse=True)
        significant_features[f'PC{i+1}'] = sorted_features
    
    return significant_features, explained_variance

# Esegui l'analisi PCA e determina le features più significative
significant_features, explained_variance = significant_features_by_pca(pca_model)


In [ ]:
print("\nExplained variance for each principal component:")
for i, ev in enumerate(explained_variance):
    print(f"PC{i+1}: {ev:.4f}")

print("\nSum of explained variance:", explained_variance.sum())

print("\nTop 10 features for each principal component:")
for pc, features in significant_features.items():
    print(f"\n{pc}:")
    for feature, contribution in features[:10]:  # Mostra le prime 10 features per ogni componente
        print(f"{feature}: {contribution:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_scree(explained_variance):
    plt.figure(figsize=(12, 4))
    plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color='b', align='center')
    plt.xlabel('Component Number')
    plt.ylabel('Variance Explained')
    plt.title('Scree Plot')
    plt.show()

def plot_cumulative_variance(explained_variance):
    cumulative_variance = np.cumsum(explained_variance)
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Variance Explained')
    plt.title('Cumulative Variance Explained by PCA')
    plt.axhline(y=0.95, color='r', linestyle='--')
    plt.text(0.5, 0.85, '95% Threshold', color='red', fontsize=12)
    plt.show()

plot_scree(explained_variance)
plot_cumulative_variance(explained_variance)


In [ ]:
from pyspark.ml.feature import PCA, VectorAssembler
from pyspark.ml import Pipeline
import pandas as pd

def PCA_loadings(pca_model, features = FEATURES, k=COMPONENTS):
    
    # Extract PCA loadings
    pca_stage = pca_model.stages[-1]
    loadings = pca_stage.pc.toArray()
    
    # Create a DataFrame for loadings
    loadings_df = pd.DataFrame(loadings, index=features, columns=[f'PC{i+1}' for i in range(k)])
    
    return loadings_df

# Run PCA on the training data
loadings_df = PCA_loadings(pca_model)

# Display PCA loadings
print(loadings_df)


In [ ]:
# Graph PCA loadings for each feature
def plot_pca_loadings(loadings_df):
    plt.figure(figsize=(24, 24))
    sns.heatmap(loadings_df, annot=True, cmap='coolwarm')
    plt.title('PCA Loadings')
    plt.show()

plot_pca_loadings(loadings_df)

In [ ]:
# Plot PCA loadings for the top 10 features
top_features = loadings_df.abs().sum(axis=1).sort_values(ascending=False).head(10).index
top_loadings_df = loadings_df.loc[top_features]
plot_pca_loadings(top_loadings_df)

In [ ]:
# Multiple Information (MI) Analysis for the features
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

# Convert Spark DataFrame to Pandas DataFrame
pandas_df = encoded_train_data.toPandas()

# Select features and target
X = pandas_df[FEATURES]
y = pandas_df[TARGET]

# Calculate mutual information
mi = mutual_info_classif(X, y)

# Display Mutual Information scores
feature_mi_scores = [(feature, score) for feature, score in zip(features, mi)]
for feature, score in feature_mi_scores:
    print(f"Feature: {feature}, MI Score: {score}")

The PCA revealed that the first two principal components explain approximately 69.17% of the variance in the dataset, with significant contributions from features such as FLOW_DURATION_MILLISECONDS, TCP_WIN_MAX_OUT, and L4_DST_PORT. The heatmap provided a visual representation of feature correlations, identifying relationships that guided feature selection. The MI analysis highlighted MAX_IP_PKT_LEN and LONGEST_FLOW_PKT as the most informative features for predicting the target variable, attack_index, indicating their potential importance in the classification tasks.

#### Feature Scalling
Feature scaling can play a crucial role in optimizing the performance of machine learning models, especially those sensitive to the scale of input data.
Applying feature scaling to the right variables essential for improving model accuracy and efficiency. It ensures that each feature contributes equally to the decision-making process, preventing models from misinterpreting the data due to arbitrary feature scales. This leads to better, more reliable predictions in multiclass classification tasks.

##### Numerical Variables with Different Scales: